In [1]:
import os
import sys
sys.path.append('../')
import numpy as np
import scipy.io as scio
import h5py
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import cv2
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.transforms.functional import adjust_sharpness
import numba
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import json

%matplotlib inline

/home/tianaoli/.conda/envs/pact/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
section_path = '/mnt/WD6TB/tianaoli/brains/'
files = os.listdir(section_path)
print(len(files))

151


Prepare edge operators.

In [3]:
laplacian = torch.tensor([[[[-1, -1, -1],
                            [-1, 9, -1],
                            [-1, -1, -1]]]], dtype=torch.float32)
sobelx = torch.tensor([[[[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]]]], dtype=torch.float32)
sobely = torch.tensor([[[[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]]]], dtype=torch.float32)

Laplacian = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding='same', bias=False)
Laplacian.weight = nn.Parameter(laplacian, requires_grad=False)
Sobelx = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding='same', bias=False)
Sobelx.weight = nn.Parameter(sobelx, requires_grad=False)
Sobely = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding='same', bias=False)
Sobely.weight = nn.Parameter(sobely, requires_grad=False)

alpha, beta = 3.4, 7.5 # Weighting parameters for first and second order derivatives.

In [4]:
info_file = 'section.json'
with open(info_file, 'r') as f:
    info = json.load(f)
section_data = info["data"]

In [5]:
# y_sec = 170
# center = [119, 130]
# for idx, file in enumerate(files):
    # if not idx == 151:
    #     continue
np.random.shuffle(section_data)    # Shuffling.
for idx, data in enumerate(section_data):
    idx_file, y_sec, center = data['idx'], data['y_sec'], data['center']
    data = scio.loadmat(os.path.join(section_path, files[idx_file]))
    brain3d = data['recondata_cut_mc_bgb']
   
    section = brain3d[:,y_sec,:]
    section = torch.tensor(section[center[0]-85:center[0]+86, center[1]-85:center[1]+86]).unsqueeze(0).unsqueeze(0)
    section = (section - section.min()) / (section.max() - section.min())
    section = F.interpolate(section, scale_factor=1.5, mode='bilinear')
    
    edge2 = Laplacian(section).abs().squeeze(0).squeeze(0)
    edge1x = Sobelx(section).squeeze(0).squeeze(0)
    edge1y = Sobely(section).squeeze(0).squeeze(0)
    edge1 = torch.sqrt(edge1x**2 + edge1y**2)
    section = section.squeeze(0).squeeze(0)
    section_sharp = section + edge1 * alpha + edge2 * beta
    section_sharp = (section_sharp - section_sharp.min()) / (section_sharp.max() - section_sharp.min())
    
    
    # Visualization
    # plt.figure(figsize=(15, 6))
    # for i in range(10):
    #     plt.subplot(2,5,i+1)
    #     y_sec = i * 10 + 110
    #     # y_sec = i * 20 + 10
    #     plt.imshow(brain3d[:,y_sec,:])
    #     plt.title(f'y_sec={y_sec}')

    fig = plt.figure(figsize=(12, 6))
    ax1 = plt.subplot(1,2,1)
    plt.imshow(section)
    plt.title(f'Section ({idx_file})', fontsize=13)
    ax2 = plt.subplot(1,2,2)
    plt.imshow(section_sharp)
    plt.title('Sharpned Image', fontsize=13)
    plt.tight_layout()
    plt.savefig(f'/mnt/WD6TB/tianaoli/dataset/Brain/sections/vis/{idx}.jpg', bbox_inches='tight')
    plt.close()
    
    # Save the images.
    np.save(f'/mnt/WD6TB/tianaoli/dataset/Brain/sections/{idx}.npy', section_sharp)